In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sklearn.linear_model as skl_lm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import roc_curve, auc
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import confusion_matrix, classification_report, precision_score
from sklearn import preprocessing
from sklearn import neighbors
from sklearn.metrics import f1_score


import statsmodels.api as sm
import statsmodels.formula.api as smf

In [4]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [18]:
# Now run the KNN Classifier on Cleaned Data

In [19]:
# Import LabelEncoder
from sklearn import preprocessing
#creating labelEncoder
le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
string_cols = ['job', 'marital','education', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']
for col in string_cols:
    df[col] = le.fit_transform(df[col])
    
    
y = df.term_deposit
X = df.drop('term_deposit', axis=1).astype('float64')

# Standardise the data 
from sklearn.preprocessing import StandardScaler

X_scaled = StandardScaler().fit_transform(X) # normalizing the features

import math
len_data = len(df)
thirty_perc = math.ceil(len_data * 0.3)
seventy_perc = math.floor(len_data * 0.7)
print(len_data)
print(thirty_perc + seventy_perc)

X_train = X_scaled[thirty_perc:]
y_train = y[thirty_perc:]

X_test = X_scaled[:thirty_perc]
y_test = y[:thirty_perc]

from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=1, weights='uniform')
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
score = clf.score(X_test, y_test)

39482
39482


In [20]:
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,term_deposit,Ethnicity_African
0,56,3,1,0,0,0,0,1,6,1,...,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0,0
1,57,7,1,3,1,0,0,1,6,1,...,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0,0
2,37,7,1,3,0,2,0,1,6,1,...,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0,0
3,40,0,1,1,0,0,0,1,6,1,...,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0,0
4,56,7,1,3,0,0,2,1,6,1,...,999,0,1,1.1,93.994,-36.4,4.857,5191.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,5,1,5,0,2,0,0,7,0,...,999,0,1,-1.1,94.767,-50.8,1.028,4963.6,1,0
41184,46,1,1,5,0,0,0,0,7,0,...,999,0,1,-1.1,94.767,-50.8,1.028,4963.6,0,0
41185,56,5,1,6,0,2,0,0,7,0,...,999,0,1,-1.1,94.767,-50.8,1.028,4963.6,0,0
41186,44,9,1,5,0,0,0,0,7,0,...,999,0,1,-1.1,94.767,-50.8,1.028,4963.6,1,0


In [21]:
summary_df = pd.DataFrame(columns = ['K', 'Accuracy', 'TPR', 'FPR', 'Precision', 'F Score'])

In [22]:
def knn(n_neighbors=1, weight='uniform'):
    clf = neighbors.KNeighborsClassifier(n_neighbors)
    clf.fit(X_train, y_train)
    pred = clf.predict(X_test)
    score = clf.score(X_test, y_test)
    score = score.round(4)
    print('K = ' + str(n_neighbors))
    #print('Accuracy Score: ' + str(score))
    #print('\n')
    print('Confusion Matrix:')
    cm_df = pd.DataFrame(confusion_matrix(y_test, pred).T, index=clf.classes_, columns=['Actual No', 'Actual Yes'])
    cm_df.index = ['Predicted No', 'Predicted Yes']
    print(cm_df)
    #print('\n')
    f_one_score = f1_score(y_test, pred)
    f_one_score = f_one_score.round(4)
    #print('F1 Score: ' + str(f_one_score))
    
    
    # Calculate TPR ----> TPR = (TP / TP + FN)
    true_pos = cm_df.iloc[1][1]
    false_neg = cm_df.iloc[0][1]
    tpr = ( true_pos / (true_pos + false_neg) ).round(4)
    
    # Calculate FPR ---> FPR = FP / (FP + TN)
    false_pos = cm_df.iloc[1][0]
    true_neg = cm_df.iloc[0][0]
    fpr = (false_pos / (false_pos + true_neg)).round(4)
    
    # Calculate Precision ---> Precision = TP / (TP + FP)
    precision =( true_pos / (true_pos + false_pos)).round(4)
    
    
    # Add results to summary table
    summary_df.loc[summary_df.shape[0]] = (n_neighbors, score, tpr, fpr, precision, f_one_score)
    
    return pred

With $k=1$, we correctly predict whether or not the customer will buy a caravan 79.1% of the time. This sounds like a fairly accurate classifier, except for the fact that we could make predictions with 96% accuracy..
Let's try adjusting the 'n_neighbors' parameter. I'll create a function to change this parameter easily.

In [23]:
k1 = knn(1)

K = 1
Confusion Matrix:
               Actual No  Actual Yes
Predicted No        9297         354
Predicted Yes       2121          73


In [24]:
k3 = knn(3)

K = 3
Confusion Matrix:
               Actual No  Actual Yes
Predicted No       10429         386
Predicted Yes        989          41


In [25]:
k5 = knn(5)

K = 5
Confusion Matrix:
               Actual No  Actual Yes
Predicted No       11061         416
Predicted Yes        357          11


In [26]:
k10 = knn(10)

K = 10
Confusion Matrix:
               Actual No  Actual Yes
Predicted No       11402         424
Predicted Yes         16           3


In [27]:
k30 = knn(30) # K=30 simply predicts everything as No, and gets a higher accuracy score

K = 30
Confusion Matrix:
               Actual No  Actual Yes
Predicted No       11418         427
Predicted Yes          0           0


<ipython-input-22-9f3fc6e77749>:31: RuntimeWarning: invalid value encountered in long_scalars
  precision =( true_pos / (true_pos + false_pos)).round(4)


In [28]:
summary_df['K'] = [1,3,5,10,30]
summary_df.set_index('K', inplace=True)
summary_df.loc[30][3] = 0.0000

In [29]:
summary_df

,Accuracy,TPR,FPR,Precision,F Score
K,,,,,
1,0.7911,0.1710,0.1858,0.0333,0.0557
3,0.8839,0.0960,0.0866,0.0398,0.0563
5,0.9347,0.0258,0.0313,0.0299,0.0277
10,0.9629,0.0070,0.0014,0.1579,0.0135
30,0.9640,0.0000,0.0000,0.0000,0.0000


In [27]:
import dataframe_image as dfi
#dfi.export(summary_df, 'KNN Summary Table1')